In [2]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [3]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['永和', '林園', '麥寮', '金門', '恆春', '桃園', '花蓮', '萬里', '線西', '美濃', '宜蘭（頭城）', '埔里', '臺東', '橋頭', '大寮', '嘉義', '士林', '三重', '土城', '淡水', '汐止', '新竹', '西屯', '關山', '馬祖', '潮州', '馬公', '林口', '大里', '南投', '頭份', '苗栗', '沙鹿', '安南', '屏東(枋山)', '湖口', '復興', '板橋', '中壢', '彰化（員林）', '三義', '大同', '高雄（湖內）', '臺南（麻豆）', '富貴角', '新營', '中山', '左營', '基隆', '屏東（琉球）', '龍潭', '古亭', '小港', '豐原', '斗六', '大城', '菜寮', '屏東', '大園', '宜蘭', '萬華', '善化', '平鎮', '陽明', '竹東', '松山', '臺西', '仁武', '忠明', '臺南', '新莊', '鳳山', '新店', '新港', '觀音', '二林', '前金', '竹山', '朴子', '彰化', '新北(樹林)', '楠梓', '冬山', '前鎮', '崙背']


In [4]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

[['2024-11-05 10:00', '屏東縣', 39, 2, '良好', 22.260899, 120.651472],
 ['2024-11-05 10:00', '屏東縣', 39, 2, '良好', 22.260899, 120.651472],
 ['2024-11-05 09:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 09:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 09:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 09:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 09:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 09:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 08:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 08:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 08:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 08:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 08:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 08:00', '屏東縣', 40, 3, '良好', 22.260899, 120.651472],
 ['2024-11-05 07:00', '屏東縣', 41, 43, '良好', 22.260899, 120.651472],
 ['2024-1

In [5]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [6]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))


In [7]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT DISTINCT sitename
    FROM records
    '''
    cursor.execute(sql)
    sitenames = []

    for items in cursor.fetchall():
        sitenames.append(items[0])
    
    print(sitenames)

['屏東(枋山)', '宜蘭（頭城）', '新北(樹林)', '屏東（琉球）', '臺南（麻豆）', '高雄（湖內）', '彰化（員林）', '大城', '富貴角', '麥寮', '關山', '馬公', '金門', '馬祖', '埔里', '復興', '永和', '竹山', '中壢', '三重', '冬山', '宜蘭', '陽明', '花蓮', '臺東', '恆春', '潮州', '屏東', '小港', '前鎮', '前金', '左營', '楠梓', '林園', '大寮', '鳳山', '仁武', '橋頭', '美濃', '臺南', '安南', '善化', '新營', '嘉義', '臺西', '朴子', '新港', '崙背', '斗六', '南投', '二林', '線西', '彰化', '西屯', '忠明', '大里', '沙鹿', '豐原', '三義', '苗栗', '頭份', '新竹', '竹東', '湖口', '龍潭', '平鎮', '觀音', '大園', '桃園', '大同', '松山', '古亭', '萬華', '中山', '士林', '淡水', '林口', '菜寮', '新莊', '板橋', '土城', '新店', '萬里', '汐止', '基隆']


In [8]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()        
    sql = '''
    SELECT date,county,aqi,pm25,status,lat,lon
    FROM records
    WHERE sitename=?
    ORDER BY date DESC;
    '''
    cursor.execute(sql,('富貴角',))
    sitename_list = [list(item) for item in cursor.fetchall()]
    print(sitename_list)

[['2024-11-05 11:00', '新北市', 64, 28, '普通', 25.29681695, 121.53656894], ['2024-11-05 10:00', '新北市', 61, 22, '普通', 25.29681695, 121.53656894], ['2024-11-05 10:00', '新北市', 61, 22, '普通', 25.29681695, 121.53656894], ['2024-11-05 10:00', '新北市', 61, 22, '普通', 25.29681695, 121.53656894], ['2024-11-05 10:00', '新北市', 61, 22, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00', '新北市', 61, 18, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00'